In [71]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from IPython.display import display

# 近軸光線追跡

レンズシステムが与えられたときに，光線高と角度を指定することで近軸光線追跡を実行する

レンズシステムの構成要素
- 屈折率
- 曲率
- 面間隔

In [226]:
class System:
    inf = 1.0e8
    def __init__(self, r_list, d_list, n_list):
        self.num = len(r_list)
        self.r = np.array([None] + r_list)
        self.d = np.array([None] + d_list + [None])
        self.n = np.array(n_list)
        self.e = np.full(self.num+1, None)
        for i in range(1, self.num):
            self.e[i] = self.d[i]/self.n[i]
        # curveture
        self.c = np.full(self.num + 1, None)
        for i in range(self.num + 1):
            if self.r[i] == 0:
                self.c[i] = 0
            elif self.r[i] is None:
                self.c[i] = None
            else:
                self.c[i] = 1/self.r[i]
        # power calc
        self.phi = np.full(self.num + 1, None)
        for i in range(1, self.num+1):
            self.phi[i] = (self.n[i]-self.n[i-1])*self.c[i]
        
        
        self.data = pd.DataFrame(data=np.vstack([self.r, self.d, self.n, self.e, self.phi]).T,
                                 columns=['r', 'd', 'n', 'e', 'phi'])
        
        result = self.parax(1, 0)
        self.bf = result['ss'][self.num]
        self.f = 1/result['u'][self.num]
        self.bo = self.bf-self.f
        
        self.inv()
        inv_result = self.parax(1, 0)
        self.ff = inv_result['ss'][self.num]
        self.inv_f = 1/inv_result['u'][self.num]
        self.fo = self.f-self.ff
        self.inv()
        
        if self.inv_f == self.f:
            print('inv_f == f')
        
        self.h = np.sum(self.d[1:-1]) + self.bf - self.ff
        
        print('system is defined as follows;')
        print(self.data)
        print(f"f={self.f:.5f}, bf={self.bf:.5f}, bo={self.bo:.5f}, "
              f"ff={self.ff:.5f}, fo ={self.fo:.5f}, h={self.h:.5f}")
    
    def parax(self, h1, u1):
        h = np.full(self.num + 1, None)
        u = np.full(self.num + 1, None)
        a = np.full(self.num + 1, None)
        s = np.full(self.num + 1, None)
        ss = np.full(self.num + 1, None)
        h[0] = h1
        u[0] = u1
        a[0] = u1*n[0]
        
        for i in range(self.num):
            s[i] = self.calc_s(h[i], u[i])
            a[i+1] = a[i] + h[i]*self.phi[i+1]
            u[i+1] = a[i+1]/n[i+1]
            ss[i+1] = self.calc_s(h[i], u[i+1])
            if i < self.num - 1:
                h[i+1] = h[i] - self.e[i+1]*a[i+1]
        
        parax_result = pd.DataFrame(data=np.vstack([h, u, a, s, ss]).T,
                                   columns=['h', 'u', 'a', 's', 'ss'])
        return parax_result
        
    def inv(self):
        inv_r = np.full(self.num+1, None)
        inv_d = np.full(self.num+1, None)
        inv_n = np.full(self.num+1, None)
        inv_e = np.full(self.num+1, None)
        inv_phi = np.full(self.num+1, None)
        inv_r[1:] = -self.r[::-1][:-1]
        inv_d = self.d[::-1]
        inv_n = self.n[::-1]
        inv_e = self.e[::-1]
        inv_phi[1:] = self.phi[::-1][:-1]
        self.r = inv_r
        self.d = inv_d
        self.n = inv_n
        self.e = inv_e
        self.phi = inv_phi
        self.data = pd.DataFrame(data=np.vstack([self.r, self.d, self.n, self.e, self.phi]).T,
                                 columns=['r', 'd', 'n', 'e', 'phi'])
    
    def calc_s(self, h, u):
        if u == 0:
            return System.inf
        else:
            return h/u

簡単な例として

|  No  |  r  |  d |  n  |
| ---- | ---- | ---- | ---- |
|  0  |  |  | 1 |
|  1  |  50  |  0  |  1.51633  |
|  2  |  0  |  | 1 |

で，h1=1, u1=0 で追跡

In [227]:
r = [50, 0]
d = [0]
n = [1, 1.51633, 1]
test = System(r, d, n)

inv_f == f
system is defined as follows;
      r     d        n     e        phi
0  None  None        1  None       None
1    50     0  1.51633     0  0.0103266
2     0  None        1  None         -0
f=96.83729, bf=96.83729, bo=0.00000, ff=96.83729, fo =0.00000, h=0.00000


In [228]:
result = test.parax(1,0)
result

,h,u,a,s,ss
0,1,0,0,1e+08,None
1,1,0.00681026,0.0103266,146.837,146.837
2,None,0.0103266,0.0103266,None,96.8373


In [229]:
r = [50, 0]
d = [0]
n = [1, 1.52621, 1]
test = System(r, d, n)

inv_f == f
system is defined as follows;
      r     d        n     e        phi
0  None  None        1  None       None
1    50     0  1.52621     0  0.0105242
2     0  None        1  None         -0
f=95.01910, bf=95.01910, bo=0.00000, ff=95.01910, fo =0.00000, h=0.00000


In [230]:
result = test.parax(1,0)
result

,h,u,a,s,ss
0,1,0,0,1e+08,None
1,1,0.00689564,0.0105242,145.019,145.019
2,None,0.0105242,0.0105242,None,95.0191


In [231]:
r = [64.26, -31.52, -262.47]
d = [2.5, 1.25]
n = [1, 1.5725, 1.6129, 1]
test = System(r, d, n)

inv_f == f
system is defined as follows;
        r     d       n         e         phi
0    None  None       1      None        None
1   64.26   2.5  1.5725   1.58983  0.00890912
2  -31.52  1.25  1.6129  0.775002 -0.00128173
3 -262.47  None       1      None  0.00233512
f=100.66681, bf=98.64449, bo=-2.02232, ff=100.31567, fo =0.35114, h=2.07882


In [233]:
result = test.parax(1,0)
result

,h,u,a,s,ss
0,1,0,0,1e+08,None
1,0.985836,0.00566558,0.00890912,174.005,176.505
2,0.979911,0.00474025,0.00764555,206.721,207.971
3,None,0.00993376,0.00993376,None,98.6445
